In [66]:
import pandas as pd
import numpy as np
from glob import glob
from nltk.corpus import stopwords
stop = stopwords.words('english')
import re

In [59]:
csvs = glob('input/tweets/*.csv')
df_tweets = pd.concat(pd.read_csv(file).assign(filename = file) for file in csvs)
df_covid = pd.read_csv('input/covid_data.csv')
df_tweets.reset_index(inplace = True)
del df_tweets['filename']
del df_tweets['index']

In [80]:
def return_hashes(col):

    return list(set([re.sub('[^\w\s]','', word) for word in col.split() if word[0] == '#']))

def return_ats(col):
    return list(set(  [re.sub('[^\w\s]','', word) for word in col.split() if word[0] == '@']  ))

def remove_stopwords(col):
    return ' '.join([word for word in col.split() if word not in stop])

In [81]:
df_tweets['text_mod'] = df_tweets['text'].str.lower() # małe znaki
df_tweets['hash'] = df_tweets['text'].apply(return_hashes) # wybieranie unikalnych hashtagów (bez punktuacji)
df_tweets['at'] = df_tweets['text'].apply(return_ats) # wybieranie unikalnych odnośników (bez punktuacji)
df_tweets['text_mod'] = df_tweets['text_mod'].str.replace('[^\w\s#@]','') # usuwanie punktuacji; można usuwać # ze zdań jeśli usunie się ten znak z regular expression
df_tweets['text_mod'] = df_tweets['text_mod'].apply(remove_stopwords)

df_tweets.created_at = pd.to_datetime(df_tweets.created_at)




## Data preview

In [82]:
print(f'Początkowa data: {min(df_tweets.created_at)}\nKońcowa data: {max(df_tweets.created_at)}\n\n')
df_tweets.info()

print('\nIlości tweetów:')
df_tweets.company.value_counts()

Początkowa data: 2019-09-26 07:00:01
Końcowa data: 2020-05-07 20:02:01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 8 columns):
company            1600 non-null object
text               1600 non-null object
created_at         1600 non-null datetime64[ns]
favourite_count    1600 non-null int64
retweet_count      1600 non-null int64
text_mod           1600 non-null object
hash               1600 non-null object
at                 1600 non-null object
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 100.1+ KB

Ilości tweetów:


BayerPharma    200
Sanofi         200
GSK            200
Merck          200
Novartis       200
AstraZeneca    200
Pfizer         200
Roche          200
Name: company, dtype: int64

In [83]:
df_tweets.head()

,company,text,created_at,favourite_count,retweet_count,text_mod,hash,at
0,AstraZeneca,"Together with partners across industry, academ...",2020-05-06 13:13:41,44,8,together partners across industry academia gov...,[COVID19],[]
1,AstraZeneca,On #GivingTuesdayNow we stand with our partner...,2020-05-05 16:27:03,32,8,#givingtuesdaynow stand partners @plan_uk @uni...,"[GivingTuesdayNow, COVID19]","[Unicef_UK, NCDAlliance, ProjectHopeorg, Plan_UK]"
2,AstraZeneca,We’re #standingtogether4asthma with patients a...,2020-05-05 12:30:15,19,7,#standingtogether4asthma patients respiratory ...,"[standingtogether4asthma, COVID19, WorldAsthma...",[WEF]
3,AstraZeneca,"Navigating asthma management is tough, especia...",2020-05-05 08:31:20,18,13,navigating asthma management tough especially ...,"[standingtogether4asthma, COVID19, WorldAsthma...",[]
4,AstraZeneca,We’re standing with the global respiratory com...,2020-05-05 07:30:01,21,5,standing global respiratory community fight #c...,"[standingtogether4asthma, COVID19, WorldAsthma...",[]
